In [2]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Text-Summarization\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Soumyadipta\\Desktop\\Text-Summarization'

In [5]:
# Entity:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

In [7]:
# Config:

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filrpath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filrpath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from textSummarization.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

[2024-03-11 12:08:30,155: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.:]
[2024-03-11 12:08:30,158: INFO: utils: NumExpr defaulting to 8 threads.:]
[2024-03-11 12:08:30,768: INFO: config: PyTorch version 1.12.1 available.:]


In [9]:
# Components:

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    
    # Convert the data into features
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [11]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()

except Exception as e:
    raise e

[2024-03-11 12:09:10,161: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-03-11 12:09:10,168: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-03-11 12:09:10,173: INFO: common: created directory at: artifacts:]
[2024-03-11 12:09:10,178: INFO: common: created directory at: artifacts/data_transformation:]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]